# Feature Engineering

## 4.1 Importing Libraries

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from scipy import stats
import warnings
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')
%matplotlib inline

## 4.2 Read Data

In [2]:
train = pd.read_csv('train2.csv')
test = pd.read_csv('test2.csv')

In [3]:
pd.set_option('display.max_columns', None)
train.head()

,Unnamed: 0,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,1,60,RL,65.0,8450,Pave,No,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,No,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,No,No,No,0,2,2008,WD,Normal,208500
1,1,2,20,RL,80.0,9600,Pave,No,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,No,No,No,0,5,2007,WD,Normal,181500
2,2,3,60,RL,68.0,11250,Pave,No,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,No,No,No,0,9,2008,WD,Normal,223500
3,3,4,70,RL,60.0,9550,Pave,No,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,No,No,No,0,2,2006,WD,Abnorml,140000
4,4,5,60,RL,84.0,14260,Pave,No,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,No,No,No,0,12,2008,WD,Normal,250000


In [4]:
nan_cols = [i for i in train.columns if train[i].isnull().any()]


In [5]:
nan_cols

[]

## 4.3 Feature Enggineering

### 4.3.1 Temporal Features

In [6]:
# We will convert into (year sold - year feature)
temp_feat = ['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold']

def tempFeat(data):
    for feature in temp_feat:
        if feature != 'YrSold':
            data[feature] = data['YrSold'] - data[feature]
    return data


In [7]:
train = tempFeat(train)
test = tempFeat(test)

In [8]:
train[['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold']].head()

,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold
0,5,5,5.0,2008
1,31,31,31.0,2007
2,7,6,7.0,2008
3,91,36,8.0,2006
4,8,8,8.0,2008


### 4.3.2 Categorical Features

For categorical features first we will replace the feature variable which occur less 1% in the data with 'rare' varible.

In [9]:
categorical_features = ['MSZoning' ,'Street' ,'Alley' ,'LotShape' ,'Utilities' ,'LotConfig' ,'Neighborhood' ,'RoofStyle' ,'RoofMatl' ,'Exterior1st' ,'Exterior2nd' ,'MasVnrType' ,'Foundation' ,'Heating' ,'CentralAir','Electrical' ,'GarageType'  ,'SaleType' ,'SaleCondition','MiscFeature','LandContour', 'LandSlope' ,'Condition1' ,'Condition2' ,'BldgType' ,'HouseStyle' ,'OverallQual' ,'OverallCond' ,'ExterQual' ,'ExterCond' ,'BsmtQual' ,'BsmtCond' ,'BsmtExposure' ,'BsmtFinType1' ,'HeatingQC' ,'FireplaceQu' ,'GarageFinish' , 'GarageQual' ,'GarageCond' ,'PoolQC' ,'Fence', 'BsmtFinType2', 'KitchenQual','PavedDrive']

In [10]:
# We will store rare features in a dict with feature_name:feature_value as pairs
rare = {}
rare_int = []
# We will replace fature values which occur less then 1% in the data with rare value
for cols in categorical_features:
    feat_dict = train[cols].value_counts().to_dict()
    for feat, vals in feat_dict.items():
        if vals/len(train) < 0.01:
            if train[cols].dtype == 'O':
                train[cols][train[cols] == feat] = "rare"
            else:
                train[cols][train[cols] == feat] = 0
                rare_int.append(cols)
            if cols in rare:
                rare[cols].append(feat)
            else:
                rare[cols] = []
                rare[cols].append(feat)

In [11]:
train.head()

,Unnamed: 0,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,1,60,RL,65.0,8450,Pave,No,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,5,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,No,Attchd,5.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,No,No,No,0,2,2008,WD,Normal,208500
1,1,2,20,RL,80.0,9600,Pave,No,Reg,Lvl,AllPub,FR2,Gtl,rare,Feedr,Norm,1Fam,1Story,6,8,31,31,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,31.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,No,No,No,0,5,2007,WD,Normal,181500
2,2,3,60,RL,68.0,11250,Pave,No,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,7,6,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,7.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,No,No,No,0,9,2008,WD,Normal,223500
3,3,4,70,RL,60.0,9550,Pave,No,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,91,36,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,8.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,No,No,No,0,2,2006,WD,Abnorml,140000
4,4,5,60,RL,84.0,14260,Pave,No,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,8,8,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,8.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,No,No,No,0,12,2008,WD,Normal,250000


Now for the test data we will stictly replace the rare feature variables which we have stored in rare dictionary.


In [12]:
for cols, var in rare.items():
    for x in var:
        if test[cols].dtype == 'O':
            test[cols].replace(to_replace = x, value = 'rare', inplace = True)
        else:
            test[cols].replace(to_replace = x, value = 0, inplace = True)

In [13]:
test.head()

,Unnamed: 0,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0,1461,20,RH,80.0,11622,Pave,No,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,49,49,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,No,Attchd,49.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,No,MnPrv,No,0,6,2010,WD,Normal
1,1,1462,20,RL,81.0,14267,Pave,No,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,52,52,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,No,Attchd,52.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,No,No,rare,12500,6,2010,WD,Normal
2,2,1463,60,RL,74.0,13830,Pave,No,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,13,12,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,13.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,No,MnPrv,No,0,3,2010,WD,Normal
3,3,1464,60,RL,78.0,9978,Pave,No,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,12,12,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,12.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,No,No,No,0,6,2010,WD,Normal
4,4,1465,120,RL,43.0,5005,Pave,No,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,18,18,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,No,Attchd,18.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,No,No,No,0,1,2010,WD,Normal


In [14]:
train.head()

,Unnamed: 0,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,1,60,RL,65.0,8450,Pave,No,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,5,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,No,Attchd,5.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,No,No,No,0,2,2008,WD,Normal,208500
1,1,2,20,RL,80.0,9600,Pave,No,Reg,Lvl,AllPub,FR2,Gtl,rare,Feedr,Norm,1Fam,1Story,6,8,31,31,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,31.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,No,No,No,0,5,2007,WD,Normal,181500
2,2,3,60,RL,68.0,11250,Pave,No,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,7,6,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,7.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,No,No,No,0,9,2008,WD,Normal,223500
3,3,4,70,RL,60.0,9550,Pave,No,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,91,36,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,8.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,No,No,No,0,2,2006,WD,Abnorml,140000
4,4,5,60,RL,84.0,14260,Pave,No,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,8,8,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,8.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,No,No,No,0,12,2008,WD,Normal,250000


### Feature Encoding

In [15]:
encoder = OneHotEncoder()

In [16]:
encoder.fit(train[categorical_features])

OneHotEncoder()

In [17]:
cat_train = encoder.transform(train[categorical_features]).toarray()
cat_test = encoder.transform(test[categorical_features]).toarray()

In [18]:
cat_train = pd.DataFrame(cat_train, columns = encoder.get_feature_names())
cat_test = pd.DataFrame(cat_test, columns = encoder.get_feature_names())

In [19]:
cat_train.head()

,x0_FV,x0_RH,x0_RL,x0_RM,x0_rare,x1_Pave,x1_rare,x2_Grvl,x2_No,x2_Pave,x3_IR1,x3_IR2,x3_Reg,x3_rare,x4_AllPub,x4_rare,x5_Corner,x5_CulDSac,x5_FR2,x5_Inside,x5_rare,x6_Blmngtn,x6_BrDale,x6_BrkSide,x6_ClearCr,x6_CollgCr,x6_Crawfor,x6_Edwards,x6_Gilbert,x6_IDOTRR,x6_MeadowV,x6_Mitchel,x6_NAmes,x6_NWAmes,x6_NoRidge,x6_NridgHt,x6_OldTown,x6_SWISU,x6_Sawyer,x6_SawyerW,x6_Somerst,x6_StoneBr,x6_Timber,x6_rare,x7_Gable,x7_Hip,x7_rare,x8_CompShg,x8_rare,x9_AsbShng,x9_BrkFace,x9_CemntBd,x9_HdBoard,x9_MetalSd,x9_Plywood,x9_Stucco,x9_VinylSd,x9_Wd Sdng,x9_WdShing,x9_rare,x10_AsbShng,x10_BrkFace,x10_CmentBd,x10_HdBoard,x10_MetalSd,x10_Plywood,x10_Stucco,x10_VinylSd,x10_Wd Sdng,x10_Wd Shng,x10_rare,x11_BrkCmn,x11_BrkFace,x11_None,x11_Stone,x11_rare,x12_BrkTil,x12_CBlock,x12_PConc,x12_Slab,x12_rare,x13_GasA,x13_GasW,x13_rare,x14_N,x14_Y,x15_FuseA,x15_FuseF,x15_SBrkr,x15_rare,x16_Attchd,x16_Basment,x16_BuiltIn,x16_Detchd,x16_No,x16_rare,x17_COD,x17_New,x17_WD,x17_rare,x18_Abnorml,x18_Family,x18_Normal,x18_Partial,x18_rare,x19_No,x19_Shed,x19_rare,x20_Bnk,x20_HLS,x20_Low,x20_Lvl,x21_Gtl,x21_Mod,x21_rare,x22_Artery,x22_Feedr,x22_Norm,x22_PosN,x22_RRAn,x22_rare,x23_Norm,x23_rare,x24_1Fam,x24_2fmCon,x24_Duplex,x24_Twnhs,x24_TwnhsE,x25_1.5Fin,x25_1Story,x25_2Story,x25_SFoyer,x25_SLvl,x25_rare,x26_0,x26_3,x26_4,x26_5,x26_6,x26_7,x26_8,x26_9,x26_10,x27_0,x27_3,x27_4,x27_5,x27_6,x27_7,x27_8,x27_9,x28_Ex,x28_Gd,x28_TA,x28_rare,x29_Fa,x29_Gd,x29_TA,x29_rare,x30_Ex,x30_Fa,x30_Gd,x30_No,x30_TA,x31_Fa,x31_Gd,x31_No,x31_TA,x31_rare,x32_Av,x32_Gd,x32_Mn,x32_No,x33_ALQ,x33_BLQ,x33_GLQ,x33_LwQ,x33_No,x33_Rec,x33_Unf,x34_Ex,x34_Fa,x34_Gd,x34_TA,x34_rare,x35_Ex,x35_Fa,x35_Gd,x35_No,x35_Po,x35_TA,x36_Fin,x36_No,x36_RFn,x36_Unf,x37_Fa,x37_No,x37_TA,x37_rare,x38_Fa,x38_No,x38_TA,x38_rare,x39_No,x39_rare,x40_GdPrv,x40_GdWo,x40_MnPrv,x40_No,x40_rare,x41_ALQ,x41_BLQ,x41_LwQ,x41_No,x41_Rec,x41_Unf,x41_rare,x42_Ex,x42_Fa,x42_Gd,x42_TA,x43_N,x43_P,x43_Y
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

### 4.3.3 Numerical Features

In [20]:
numerical = [ 'MasVnrArea' , '1stFlrSF' , '2ndFlrSF' , 'LowQualFinSF' , 'GrLivArea' , 'FullBath' , 'HalfBath' , 'BedroomAbvGr' , 'KitchenAbvGr' , 'Fireplaces' , 'GarageYrBlt' , 'GarageArea' ,'GarageCars' , 'WoodDeckSF' , 'OpenPorchSF' , 'EnclosedPorch' , '3SsnPorch' , 'ScreenPorch' , 'PoolArea' , 'MoSold' , 'YrSold','LotFrontage', 'LotArea', 'BsmtFinSF1','BsmtFinSF2','BsmtUnfSF', 'BsmtFullBath','TotalBsmtSF','BsmtHalfBath','TotRmsAbvGrd']

### Feature Scaling

In [21]:
scaler = MinMaxScaler()

In [22]:
scaler.fit(train[numerical])

MinMaxScaler()

In [23]:
num_train = scaler.transform(train[numerical])
num_test = scaler.transform(test[numerical])

In [24]:
num_train = pd.DataFrame(num_train, columns = numerical)
num_test = pd.DataFrame(num_test, columns = numerical)

In [25]:
num_train.head(25)

,MasVnrArea,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageYrBlt,GarageArea,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,YrSold,LotFrontage,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,BsmtFullBath,TotalBsmtSF,BsmtHalfBath,TotRmsAbvGrd
0,0.122500,0.119780,0.413559,0.0,0.259231,0.666667,0.5,0.375,0.333333,0.000000,0.002488,0.386460,0.50,0.000000,0.111517,0.000000,0.000000,0.000000,0.0,0.090909,0.50,0.207668,0.033420,0.125089,0.000000,0.064212,0.333333,0.140098,0.0,0.500000
1,0.000000,0.212942,0.000000,0.0,0.174830,0.666667,0.0,0.375,0.333333,0.333333,0.015423,0.324401,0.50,0.347725,0.000000,0.000000,0.000000,0.000000,0.0,0.363636,0.25,0.255591,0.038795,0.173281,0.000000,0.121575,0.000000,0.206547,0.5,0.333333
2,0.101250,0.134465,0.419370,0.0,0.273549,0.666667,0.5,0.375,0.333333,0.333333,0.003483,0.428773,0.50,0.000000,0.076782,0.000000,0.000000,0.000000,0.0,0.727273,0.50,0.217252,0.046507,0.086109,0.000000,0.185788,0.333333,0.150573,0.0,0.333333
3,0.000000,0.143873,0.366102,0.0,0.260550,0.333333,0.0,0.375,0.333333,0.333333,0.003980,0.452750,0.75,0.000000,0.063985,0.492754,0.000000,0.000000,0.0,0.090909,0.00,0.191693,0.038561,0.038271,0.000000,0.231164,0.333333,0.123732,0.0,0.416667
4,0.218750,0.186095,0.509927,0.0,0.351168,0.666667,0.5,0.500,0.333333,0.333333,0.003980,0.589563,0.75,0.224037,0.153565,0.000000,0.000000,0.000000,0.0,1.000000,0.50,0.268371,0.060576,0.116052,0.000000,0.209760,0.333333,0.187398,0.0,0.583333
5,0.000000,0.106012,0.274092,0.0,0.193670,0.333333,0.5,0.125,0.333333,0.000000,0.007960,0.338505,0.50,0.046674,0.054845,0.000000,0.629921,0.000000,0.0,0.818182,0.75,0.271565,0.059899,0.129695,0.000000,0.027397,0.333333,0.130278,0.0,0.250000
6,0.116250,0.312070,0.000000,0.0,0.256217,0.666667,0.0,0.375,0.333333,0.333333,0.001493,0.448519,0.50,0.297550,0.104205,0.000000,0.000000,0.000000,0.0,0.636364,0.25,0.239617,0.041057,0.242558,0.000000,0.135702,0.333333,0.275941,0.0,0.416667
7,0.150000,0.177375,0.476029,0.0,0.330821,0.666667,0.5,0.375,0.333333,0.666667,0.017910,0.341326,0.50,0.274212,0.372943,0.413043,0.000000,0.000000,0.0,0.909091,0.75,0.000000,0.042450,0.152197,0.021710,0.092466,0.333333,0.181178,0.0,0.416667
8,0.000000,0.157871,0.364165,0.0,0.271289,0.666667,0.0,0.250,0.666667,0.666667,0.038308,0.330042,0.50,0.105018,0.000000,0.371377,0.000000,0.000000,0.0,0.272727,0.50,0.162939,0.022529,0.000000,0.000000,0.407534,0.000000,0.155810,0.0,0.500000
9,0.000000,0.170491,0.000000,0.0,0.139977,0.333333,0.0,0.250,0.666667,0.666667,0.034328,0.144570,0.25,0.000000,0.007313,0.000000,0.000000,0.000000,0.0,0.000000,0.50,0.159744,0.028605,0.150780,0.000000,0.059932,0.333333,0.162193,0.0,0.250000


### Feature Normization

In [27]:
num_features=['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea']

In [29]:
for feature in num_features:
    num_train[feature] = train[feature].apply(lambda x: np.log(x) if x != 0 else x)
    num_test[feature] = test[feature].apply(lambda x: np.log(x) if x != 0 else x)
    # num_train[feature] = np.log(train[feature])
    # num_test[feature] = np.log(test[feature])

In [30]:
xtrain = pd.concat([cat_train, num_train, train['SalePrice']], axis=1)
xtest = pd.concat([cat_test, num_test], axis=1)

In [31]:
xtrain.to_csv('xtrain.csv')
xtest.to_csv('xtest.csv')

In [35]:
xtest.LotFrontage[xtest.LotFrontage == '-inf'].count()

0

In [37]:
xtrain.tail()

,x0_FV,x0_RH,x0_RL,x0_RM,x0_rare,x1_Pave,x1_rare,x2_Grvl,x2_No,x2_Pave,x3_IR1,x3_IR2,x3_Reg,x3_rare,x4_AllPub,x4_rare,x5_Corner,x5_CulDSac,x5_FR2,x5_Inside,x5_rare,x6_Blmngtn,x6_BrDale,x6_BrkSide,x6_ClearCr,x6_CollgCr,x6_Crawfor,x6_Edwards,x6_Gilbert,x6_IDOTRR,x6_MeadowV,x6_Mitchel,x6_NAmes,x6_NWAmes,x6_NoRidge,x6_NridgHt,x6_OldTown,x6_SWISU,x6_Sawyer,x6_SawyerW,x6_Somerst,x6_StoneBr,x6_Timber,x6_rare,x7_Gable,x7_Hip,x7_rare,x8_CompShg,x8_rare,x9_AsbShng,x9_BrkFace,x9_CemntBd,x9_HdBoard,x9_MetalSd,x9_Plywood,x9_Stucco,x9_VinylSd,x9_Wd Sdng,x9_WdShing,x9_rare,x10_AsbShng,x10_BrkFace,x10_CmentBd,x10_HdBoard,x10_MetalSd,x10_Plywood,x10_Stucco,x10_VinylSd,x10_Wd Sdng,x10_Wd Shng,x10_rare,x11_BrkCmn,x11_BrkFace,x11_None,x11_Stone,x11_rare,x12_BrkTil,x12_CBlock,x12_PConc,x12_Slab,x12_rare,x13_GasA,x13_GasW,x13_rare,x14_N,x14_Y,x15_FuseA,x15_FuseF,x15_SBrkr,x15_rare,x16_Attchd,x16_Basment,x16_BuiltIn,x16_Detchd,x16_No,x16_rare,x17_COD,x17_New,x17_WD,x17_rare,x18_Abnorml,x18_Family,x18_Normal,x18_Partial,x18_rare,x19_No,x19_Shed,x19_rare,x20_Bnk,x20_HLS,x20_Low,x20_Lvl,x21_Gtl,x21_Mod,x21_rare,x22_Artery,x22_Feedr,x22_Norm,x22_PosN,x22_RRAn,x22_rare,x23_Norm,x23_rare,x24_1Fam,x24_2fmCon,x24_Duplex,x24_Twnhs,x24_TwnhsE,x25_1.5Fin,x25_1Story,x25_2Story,x25_SFoyer,x25_SLvl,x25_rare,x26_0,x26_3,x26_4,x26_5,x26_6,x26_7,x26_8,x26_9,x26_10,x27_0,x27_3,x27_4,x27_5,x27_6,x27_7,x27_8,x27_9,x28_Ex,x28_Gd,x28_TA,x28_rare,x29_Fa,x29_Gd,x29_TA,x29_rare,x30_Ex,x30_Fa,x30_Gd,x30_No,x30_TA,x31_Fa,x31_Gd,x31_No,x31_TA,x31_rare,x32_Av,x32_Gd,x32_Mn,x32_No,x33_ALQ,x33_BLQ,x33_GLQ,x33_LwQ,x33_No,x33_Rec,x33_Unf,x34_Ex,x34_Fa,x34_Gd,x34_TA,x34_rare,x35_Ex,x35_Fa,x35_Gd,x35_No,x35_Po,x35_TA,x36_Fin,x36_No,x36_RFn,x36_Unf,x37_Fa,x37_No,x37_TA,x37_rare,x38_Fa,x38_No,x38_TA,x38_rare,x39_No,x39_rare,x40_GdPrv,x40_GdWo,x40_MnPrv,x40_No,x40_rare,x41_ALQ,x41_BLQ,x41_LwQ,x41_No,x41_Rec,x41_Unf,x41_rare,x42_Ex,x42_Fa,x42_Gd,x42_TA,x43_N,x43_P,x43_Y,MasVnrArea,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageYrBlt,GarageArea,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,YrSold,LotFrontage,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,BsmtFullBath,TotalBsmtSF,BsmtHalfBath,TotRmsAbvGrd,SalePrice
1455,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,6.859615,0.336077,0.0,7.406711,0.666667,0.5,0.375,0.333333,0.333333,0.003980,0.324401,0.50,0.000000,0.073126,0.000000,0.0,0.0,0.0,0.636364,0.25,4.127134,8.976768,0.000000,0.000000,0.407962,0.000000,0.155974,0.0,0.416667,175000
1456,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0